In [1]:
# document_loaders를 통해 다양한 문서를 읽어올 수 있다.
from langchain.llms.ollama import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = Ollama(
  model="llama3:latest",
  temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=200, # 각 청크의 최대 길이(200개의 문자)
  chunk_overlap=50 # 청크간의 중첩 길이(이전 50개의 문자)
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OllamaEmbeddings(
  base_url="http://localhost:11434",  # 모델 서버의 URL
  model="llama3:latest"
)

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)


vectorstore = Chroma.from_documents(docs, cache_embeddings)

chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="refine",
  retriever=vectorstore.as_retriever()
)

chain.run("카카오페이머니카드의 출시일은 언제입니까?")

/Users/hooni/Documents/github/fullstack-gpt/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/hooni/Documents/github/fullstack-gpt/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 211, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 217, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200
/Users/hooni/Documents/github/fullstack-gpt/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method 

"With the additional context, we can still conclude that the passage does not provide insight into KakaoPay's equity structure or ownership.\n\nHowever, we can note that the passage mentions the addition of offline card payment methods such as Payco/SSG Pay in August 2019, which was initiated by SPC Group and its affiliates. This information is relevant to KakaoPay's services and business development, but it does not provide any information about the company's equity structure or ownership.\n\nTherefore, our refined answer remains:\n\n카카오페이의 지분구조는 언급되지 않습니다. (The equity structure of KakaoPay is not mentioned.)"

In [2]:

from langchain_core.messages.tool import tool_call_chunk

llm.invoke("Hello, world!")

"Hello there! It's great to meet you! Welcome to the world of AI-powered conversations. I'm here to help answer your questions, provide information, and have some fun with you. What brings you to this corner of the internet today?"

In [3]:
# 유사도 검색
vectorstore.similarity_search("카카오페이카드는 언제 출시되었습니까?")



[Document(metadata={'source': './files/chapter_one.txt'}, page_content='2019년 8월 8일에는 기존 카카오페이머니 결제 외에 페이코/SSG페이같은 오프라인 카드 결제도 추가했으며, SPC그룹 계열 등을 시작으로 시행했다.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='카카오페이머니는 등록한 계좌에서 대부분 1만 원 이상 충전이 가능하지만, NH농협은행에서 나오는 "NH X 카카오페이 통장"의 경우는 1만 원 미만의 금액도 카카오페이머니 충전이 가능하다.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='2018년에는 QR코드 결제도 추가했는데, 바코드를 통해 곧바로 카카오페이머니로 결제하는 오프라인 매장결제와 달리 QR코드를 스캔한 후 이미 충전해 놓은 카카오페이머니를 송금하는 방식이다.'),
 Document(metadata={'source': './files/chapter_one.txt'}, page_content='2018년 1월 10일에는 선불카드인 "카카오페이카드"를 출시했다. 카카오페이카드는 캐릭터가 라이언 밖에 없다.')]